# Replay strategies


In [2]:
import networkx as nx
from tqdm.auto import tqdm

from caveclient import CAVEclient
from paleo import (
    get_initial_graph,
    get_operations_level2_edits,
    get_nucleus_supervoxel,
    get_node_aliases,
    get_detailed_change_log,
)

In [3]:
root_id = 864691135639556411
client = CAVEclient("minnie65_public", version=1078)
change_log = get_detailed_change_log(root_id, client, filtered=False)
change_log

,user_id,before_root_ids,after_root_ids,is_merge,in_neuron,is_relevant,user_name,user_affiliation,added_edges,roots,...,source_coords,timestamp,user,bb_offset,sink_ids,source_ids,operation_exception,operation_status,operation_ts,removed_edges
operation_id,,,,,,,,,,,,,,,,,,,,,
9028,161,"[864691135338600033, 864691135349337074]",[864691134884740858],True,False,True,Forrest Collman,,"[[86214631483474790, 86214631483480364]]",[864691134884740858],...,"[[77976, 78528, 19617]]",2020-07-16 17:06:40.476000+00:00,161,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14768,161,"[864691135437585906, 864691136156871317]",[864691136272932030],True,False,True,Forrest Collman,,"[[82977807025951488, 82977807025951510]]",[864691136272932030],...,"[[66096, 79024, 22345]]",2020-07-17 05:27:27.308000+00:00,161,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25672,161,"[864691135190922026, 864691135566303299]",[864691135493963920],True,False,True,Forrest Collman,,"[[102399030327517695, 102399030327517713]]",[864691135493963920],...,"[[136888, 77104, 19675]]",2020-07-18 03:13:03.205000+00:00,161,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26481,161,"[864691134843526581, 864691135573183940]",[864691135415463098],True,False,True,Forrest Collman,,"[[82769106176487530, 82769106176488477]]",[864691135415463098],...,"[[65352, 88056, 23926]]",2020-07-18 04:56:11.137000+00:00,161,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30000,161,"[864691134975250238, 864691135587889860]",[864691135476114088],True,False,True,Forrest Collman,,"[[84114358854033172, 84114358854042596]]",[864691135476114088],...,"[[70176, 118784, 25120]]",2020-07-18 12:24:14.786000+00:00,161,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532312,2378,[864691135475323712],[864691137054392182],False,True,True,Swagat Pal,Ariadne,NaN,"[864691136310375258, 864691137054392182]",...,"[[101888, 88418, 19732]]",2023-01-16 07:15:31.357000+00:00,2378,"[240, 240, 24]",[92761536032260805],[92761536032253267],,0,2023-01-16 07:15:31.357000+00:00,"[[92761536032252020, 92761536032253864], [9276..."
532314,2378,[864691137054392182],[864691136089721911],False,True,True,Swagat Pal,Ariadne,NaN,"[864691136371425416, 864691136089721911]",...,"[[81310, 89741, 20788]]",2023-01-16 07:16:07.087000+00:00,2378,"[240, 240, 24]",[87202817693302214],[87132448949123115],,0,2023-01-16 07:16:07.087000+00:00,"[[87132448949123115, 87132448949123137], [8713..."
532327,2378,[864691136089721911],[864691135975162223],False,True,True,Swagat Pal,Ariadne,NaN,"[864691136100569717, 864691135975162223]",...,"[[89915, 82579, 20415]]",2023-01-16 07:17:05.243000+00:00,2378,"[240, 240, 24]",[89523062038866888],[89523062038871240],,0,2023-01-16 07:17:05.243000+00:00,"[[89523062038866822, 89523062038869094], [8952..."


In [4]:
networkdeltas = get_operations_level2_edits(change_log.index, client)
initial_graph = get_initial_graph(root_id, client)
nuc_supervoxel_id = get_nucleus_supervoxel(root_id, client)
node_info = get_node_aliases(nuc_supervoxel_id, client, stop_layer=2)
anchor_nodes = node_info.index

Extracting level2 edits:   0%|          | 0/693 [00:00<?, ?it/s]

/Users/ben.pedigo/code/cave-edits/.venv/lib/python3.12/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


Getting initial graph:   0%|          | 0/309 [00:00<?, ?it/s]

In [5]:
from paleo import get_metaedits

metaedits, operation_map = get_metaedits(networkdeltas)

In [6]:
change_log["metaedit"] = change_log.index.map(operation_map)

metaedit_info = (
    change_log.groupby("metaedit")
    .agg({"timestamp": "min", "is_merge": "any", "metaedit": "count"})
    .rename(
        columns={
            "metaedit": "n_edits",
            "timestamp": "first_timestamp",
            "is_merge": "has_merge",
        }
    )
)
metaedit_info["operation_ids"] = (
    change_log.reset_index().groupby("metaedit")["operation_id"].unique()
)
metaedit_info = metaedit_info.sort_values("first_timestamp")
metaedit_info

,first_timestamp,has_merge,n_edits,operation_ids
metaedit,,,,
0,2020-07-16 17:06:40.476000+00:00,True,1,[9028]
1,2020-07-17 05:27:27.308000+00:00,True,1,[14768]
2,2020-07-18 03:13:03.205000+00:00,True,1,[25672]
3,2020-07-18 04:56:11.137000+00:00,True,1,[26481]
4,2020-07-18 12:24:14.786000+00:00,True,1,[30000]
...,...,...,...,...
358,2023-01-16 07:14:29.124000+00:00,False,1,[532299]
359,2023-01-16 07:15:31.357000+00:00,False,1,[532312]
360,2023-01-16 07:16:07.087000+00:00,False,1,[532314]


In [7]:
metaedit_info.index.is_monotonic_increasing

True

In [8]:
from paleo import resolve_edit

graph = initial_graph.copy()

components = []
subgraphs = []

# remember to include the initial state
edits = metaedits.copy()
edits = {-1: None, **edits}

# after each edit, apply it and store the connected component for the nucleus node
for edit_id, delta in tqdm(edits.items(), disable=False):
    component = resolve_edit(graph, delta, node_info.index)
    components.append(component)

    subgraph = graph.subgraph(component).copy()
    subgraphs.append(subgraph)

  0%|          | 0/364 [00:00<?, ?it/s]

In [9]:
final_edgelist = client.chunkedgraph.level2_chunk_graph(root_id)
final_graph = nx.from_edgelist(final_edgelist)

In [10]:
import numpy as np

print(np.setdiff1d(final_graph.nodes, subgraphs[-1].nodes))

[]


In [14]:
used_l2_ids = np.unique(np.concatenate([list(c) for c in components]))

l2data = client.l2cache.get_l2data_table(used_l2_ids, split_columns=True)
l2data

In [16]:
import pandas as pd

from pcg_skel import pcg_skeleton_direct

nuc_table = client.info.get_datastack_info()["soma_table"]
nuc_info = client.materialize.query_table(
    nuc_table, filter_equal_dict=dict(pt_root_id=root_id)
)
nuc_loc = nuc_info["pt_position"].values[0]


def skeletonize(subgraph):
    component = pd.Index(list(subgraph.nodes))
    vertices = l2data.loc[
        component, ["rep_coord_nm_x", "rep_coord_nm_y", "rep_coord_nm_z"]
    ].values
    edges = nx.to_pandas_edgelist(subgraph).values
    edges = np.vectorize(component.get_loc)(edges)
    skeleton = pcg_skeleton_direct(vertices, edges, root_point=nuc_loc)
    return skeleton


skeletons = {}
for subgraph in tqdm(subgraphs, desc="Skeletonizing"):
    skeleton = skeletonize(subgraph)
    skeletons[subgraph] = skeleton


# graph = initial_graph.copy()
# # keep track of components that are reached as we go
# components = []
# # remember to include the initial state
# metaedits = {-1: None, **metaedits}

# skeletons = {}
# # after each edit, apply it and store the connected component for the nucleus node
# for edit_id, delta in tqdm(metaedits.items(), disable=False):
#     component = resolve_edit(graph, delta, node_info.index)
#     skeleton = skeletonize(graph, component)
#     skeletons[edit_id] = skeleton

Skeletonizing:   0%|          | 0/364 [00:00<?, ?it/s]

In [18]:
import pyvista as pv

plotter = pv.Plotter()
plotter.open_gif("skeleton_evolution.gif", fps=30)


def skel_to_poly(skeleton):
    vertices = skeleton.vertices
    edges = skeleton.edges
    lines = np.full((len(edges), 3), 2)
    lines[:, 1:] = edges
    line_poly = pv.PolyData(vertices, lines=lines)
    return line_poly


last_skeleton = skeletons[list(skeletons.keys())[-1]]
actor = plotter.add_mesh(skel_to_poly(last_skeleton), color="black", line_width=2)
plotter.write_frame()
plotter.remove_actor(actor)

first_skeleton = skeletons[list(skeletons.keys())[0]]
actor = plotter.add_mesh(skel_to_poly(first_skeleton), color="blue", line_width=5)

for edit_id, skeleton in skeletons.items():
    line_poly = skel_to_poly(skeleton)
    actor = plotter.add_mesh(line_poly, color="black", line_width=2)
    plotter.write_frame()
    plotter.remove_actor(actor)

plotter.close()

In [19]:
import time
from pcg_skel import pcg_skeleton

currtime = time.time()
scratch_skeleton = pcg_skeleton(
    root_id,
    client,
    root_point=nuc_loc,
    root_point_resolution=[1, 1, 1],
)

In [20]:
from paleo import get_all_time_synapses

pre_synapses, post_synapses = get_all_time_synapses(root_id, client, verbose=True)

Getting original roots took 2.19 seconds
Getting latest roots took 5.79 seconds
Querying synapse table for pre-synapses...
Querying synapse table took 37.88 seconds
Querying synapse table for post-synapses...
Querying synapse table took 269.79 seconds


In [21]:
# for all supervoxels, look up their level2 node now
supervoxel_ids = list(pre_synapses["pre_pt_supervoxel_id"])
current_ts = client.timestamp

node_ids = client.chunkedgraph.get_roots(
    supervoxel_ids, stop_layer=2, timestamp=current_ts
)
node_to_supervoxel = pd.Series(
    index=node_ids, data=supervoxel_ids, name="supervoxel_id"
)

# supervoxel_l2 = (
#     pd.Series(index=supervoxel_ids.values, data=node_ids, name="l2_id")
#     .reset_index()
#     .rename(columns={"index": "supervoxel_id"})
# )
# supervoxel_l2
node_to_supervoxel

176353695112364040    104296101074437759
155389031826326336     83331437788956885
175512087747887932    103454493710433024
176985433127780785    104927839090261268
175090218880205152    103032624842673942
                             ...        
151038607241511282     78981013203931372
154486882133082666     82429288095659048
148853328082633067     76795734044942461
151038469802558109     78980875764981044
151247995755560974     79190401717703815
Name: supervoxel_id, Length: 62372, dtype: int64

In [128]:
# find out when they were made

timestamps = client.chunkedgraph.get_root_timestamps(node_ids)

In [ ]:
# anything that was created at before oldest_ts we can ignore
oldest_ts = client.chunkedgraph.get_oldest_timestamp()

timestamps = pd.Series(timestamps, index=node_ids).sort_values()
(timestamps < oldest_ts).mean()

In [ ]:
len(new_nodes.intersection(used_l2_ids))

In [ ]:
new_nodes = timestamps[timestamps > oldest_ts].index
new_nodes

In [ ]:
supervoxels_to_lookup = np.squeeze(
    supervoxel_l2.set_index("l2_id").loc[new_nodes].values
)

from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib
from paleo import get_node_aliases


with tqdm_joblib(total=len(supervoxels_to_lookup)) as pbar:
    historical_l2_ids = Parallel(n_jobs=-1)(
        delayed(lambda sv: get_node_aliases(sv, client).index.to_list())(sv)
        for sv in supervoxels_to_lookup
    )

In [143]:
supervoxel_historical_l2_ids = {
    sv: l2s for sv, l2s in zip(supervoxels_to_lookup, historical_l2_ids)
}

In [ ]:
relevant_historical_l2_ids = {}
for sv, historical_l2_ids in supervoxel_historical_l2_ids.items():
    relevant = np.intersect1d(historical_l2_ids, used_l2_ids)
    if len(relevant) > 0:
        relevant_historical_l2_ids[sv] = relevant
print(len(relevant_historical_l2_ids))

In [ ]:
relevant_historical_l2_ids

In [22]:
from paleo import get_nodes_aliases

out = get_nodes_aliases(supervoxel_ids, client)
out

  0%|          | 0/2253 [00:00<?, ?it/s]

{84103500505983938: [156161094543475646,
  156161094543475645,
  156161094543475305],
 82054766947264972: [154112360984872492, 154112360984871737],
 85714147802809649: [157771741840541395, 157771741840540037],
 85714147802765811: [157771741840541395, 157771741840539961],
 91702500780215832: [163760094817813700, 163760094817813101],
 87968833498931437: [160026427536704785, 160026427536703824],
 89237326387266319: [161294920424883396,
  161294920424883395,
  161294920424882749],
 106765534330555820: [178823128368022448, 178823128368022168],
 89028831830419515: [161086425868010745, 161086425868010050],
 88901220366149681: [160958814403691535, 160958814403690985],
 88901220366042277: [160958814403691535, 160958814403690985],
 88971589177095484: [161029183214977088, 161029183214977025],
 88971589177065611: [161029183214977088, 161029183214977025],
 89942044487070178: [161999638524593194, 161999638524592839],
 92265862916005157: [164323456953877461,
  164323456953877459,
  164323456953877457

In [176]:
stop_layer = 2
# for all supervoxels, look up their level2 node now
current_ts = client.timestamp
node_ids = client.chunkedgraph.get_roots(
    supervoxel_ids, stop_layer=stop_layer, timestamp=current_ts
)
node_to_supervoxel = pd.Series(index=node_ids, data=supervoxel_ids)
node_to_supervoxel.index.name = "node_id"
node_to_supervoxel.name = "supervoxel_id"

# find out when they were made
timestamps = client.chunkedgraph.get_root_timestamps(node_ids)
node_timestamps = pd.Series(data=timestamps, index=node_ids)
node_timestamps.index.name = "node_id"
node_timestamps.name = "timestamp"

In [177]:
# anything that was created at before oldest_ts we can ignore
oldest_ts = client.chunkedgraph.get_oldest_timestamp()

# everything else, we need to look up its history
new_nodes = node_timestamps[node_timestamps > oldest_ts].index
supervoxels_to_lookup = node_to_supervoxel.loc[new_nodes]

In [ ]:
from datetime import datetime
from datetime import timedelta

TIMESTAMP_DELTA = timedelta(microseconds=1)

possible_timestamps = change_log["timestamp"]
possible_timestamps = [datetime.fromisoformat(ts) for ts in possible_timestamps.values]
outs = []
for ts in tqdm(possible_timestamps):
    out = client.chunkedgraph.get_roots(
        supervoxels_to_lookup, stop_layer=2, timestamp=ts
    )
    outs.append(out)

from joblib import Parallel, delayed

# with tqdm_joblib(total=len(possible_timestamps)):
#     outs = Parallel(n_jobs=-1)(
#         delayed(client.chunkedgraph.get_roots)(
#             supervoxels_to_lookup, stop_layer=2, timestamp=ts - TIMESTAMP_DELTA
#         )
#         for ts in possible_timestamps
#     )

In [ ]:
node_ids_by_time = pd.DataFrame(np.array(outs), index=possible_timestamps).T
node_ids_by_time.index = supervoxels_to_lookup
node_ids_by_time

In [ ]:
current_ts = created_ts - TIMESTAMP_DELTA
node_id = client.chunkedgraph.get_roots(
    supervoxel_id, stop_layer=stop_layer, timestamp=current_ts
)[0]

In [ ]:
timestamps.loc[new_nodes]

In [ ]:
def get_level2_lineage_components(networkdeltas_by_operation):
    graph = nx.DiGraph()

    for operation_id, delta in networkdeltas_by_operation.items():
        for node1 in delta.removed_nodes:
            for node2 in delta.added_nodes:
                graph.add_edge(node1, node2, operation_id=operation_id)

    level2_lineage_components = list(nx.weakly_connected_components(graph))

    level2_lineage_component_map = {}
    for i, component in enumerate(level2_lineage_components):
        for node in component:
            level2_lineage_component_map[node] = i

    level2_lineage_component_map = pd.Series(level2_lineage_component_map)

    return level2_lineage_component_map


level2_id_components = get_level2_lineage_components(networkdeltas)
level2_id_components

In [75]:
new_nodes = timestamps[timestamps > oldest_ts].index

In [ ]:
new_nodes.intersection(level2_id_components.index)

In [ ]:
plotter = pv.Plotter(shape=(1, 2))

last_skeleton = skeletons[list(skeletons.keys())[-1]]
plotter.subplot(0, 0)
plotter.add_mesh(skel_to_poly(last_skeleton), color="black", line_width=2)

plotter.subplot(0, 1)
plotter.add_mesh(skel_to_poly(scratch_skeleton), color="black", line_width=2)

plotter.show()